In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
kf = KFold(n_splits=5)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
def modelo_teste():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    model.summary()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["sparse_categorical_accuracy"])
    return model

In [ ]:
for train_index, test_index in kf.split(x_train):

    tmp_trainx, tmp_trainY, tmp_testx, tmp_testy = x_train[train_index], y_train[train_index], x_train[test_index], y_train[test_index]

    train_generator = datagen.flow(tmp_trainx,
                               tmp_trainY,
                               batch_size=64,
                               shuffle=True,
                               seed=1010)
    
    test_k_fold_generator = datagen.flow(tmp_testx,
                               tmp_testy,
                               batch_size=64,
                               shuffle=False)
    
    model = modelo_teste()

    # Treina a rede neural
    model.fit(train_generator,epochs=10)

    y_predicted = []
    pred = model.predict(test_k_fold_generator)
    for p in pred:
        y_predicted.append(p.argmax())

    from sklearn import metrics
    # Printa a matriz de confusão
    print(metrics.confusion_matrix(tmp_testy, y_predicted))
    # Printa as métricas precisão, recall e F1-score
    print(metrics.classification_report(tmp_testy, y_predicted, digits=3))

    tf.keras.backend.clear_session()